<a href="https://colab.research.google.com/github/adrianmoses/text-search-nlp/blob/main/SentenceSemanticSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch faiss-gpu sentence-transformers

     |████████████████████████████████| 3.1 MB 28.0 MB/s 
     |████████████████████████████████| 89.7 MB 11 kB/s 
     |████████████████████████████████| 78 kB 6.7 MB/s 
     |████████████████████████████████| 895 kB 36.2 MB/s 
     |████████████████████████████████| 596 kB 49.8 MB/s 
     |████████████████████████████████| 59 kB 5.9 MB/s 
     |████████████████████████████████| 3.3 MB 36.0 MB/s 
     |████████████████████████████████| 1.2 MB 47.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=5b357a656eb1777b01520f1d85480a18c83e183733e083e993c2ae48f5fcb033
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import faiss
import torch
from sentence_transformers import SentenceTransformer
import json

In [3]:
with open('drive/MyDrive/wiki_health_data.json') as f:
  data = json.load(f)


In [5]:
model = SentenceTransformer("distilbert-base-nli-stsb-mean-tokens")

Downloading:   0%|          | 0.00/345 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/555 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/505 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
corpus = [d['text'] for d in data]
corpus_embeddings = model.encode(corpus)

In [17]:
import numpy as np
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))

index.add_with_ids(
    corpus_embeddings,
    np.array(range(0, len(data)))
)

In [20]:
def search(query: str, k=1):
    encoded_query = model.encode([query])
    top_k = index.search(encoded_query, k)
    scores = top_k[0][0]
    results = [data[_id] for _id in top_k[1][0]]
    return list(zip(results, scores))

In [21]:
search("symptoms of swine flu", k=5)

[({'text': 'Swine influenza is an infection caused by any one of several types of swine influenza viruses. Swine influenza virus (SIV) or swine-origin influenza virus (S-OIV) is any strain of the influenza family of viruses that is endemic in pigs. As of 2009, the known SIV strains include influenza C and the subtypes of  influenza A known as H1N1, H1N2, H2N1, H3N1, H3N2, and H2N3.\nSwine influenza virus is common throughout pig populations worldwide. Transmission of the virus from pigs to humans is not common and does not always lead to human flu, often resulting only in the production of antibodies in the blood. If transmission does cause human flu, it is called zoonotic swine flu. People with regular exposure to pigs are at increased risk of swine flu infection.\nAround the mid-20th century, identification of influenza subtypes became possible, allowing accurate diagnosis of transmission to humans. Since then, only 50 such transmissions have been confirmed. These strains of swine fl

In [22]:
faiss.write_index(index, 'cdc_search.index')